In [ ]:
# | default_exp _components.client

Note: 

While writing doc strings, please use the below syntax for linking methods/classes. So that the methods/classes gets highlighted in the browser and clicking on it will take the user to the linked function

    - To link a method from the class same file please use the `method_name` format.
    - To link a method from a different Class (can in a seperate file also) please use `Classname.method_name` format.

In [ ]:
from airt._testing import activate_by_import

[INFO] airt._testing.activate_by_import: Testing environment activated.


In [ ]:
# | export

from typing import *

In [ ]:
# | exporti

import importlib
import json
import os
import secrets
import urllib.parse

import pandas as pd
from fastcore.foundation import patch

import airt
from airt._constant import (
    CLIENT_NAME,
    SERVER_URL,
    SERVICE_PASSWORD,
    SERVICE_TOKEN,
    SERVICE_USERNAME,
)
from airt._helper import delete_data, get_base_url, get_data, post_data, export
from airt._logger import get_logger, set_level

In [ ]:
import logging
from contextlib import contextmanager
from datetime import datetime, timedelta
from random import randrange

import pytest

import airt._sanitizer
from airt._components.user import User
from airt._constant import SERVICE_PASSWORD, SERVICE_SUPER_USER, SERVICE_USERNAME
from airt._docstring.helpers import run_examples_from_docstring

In [ ]:
# | exporti

logger = get_logger(__name__)

In [ ]:
display(logger.getEffectiveLevel())
assert logger.getEffectiveLevel() == logging.INFO

logger.debug("This is a debug message")
logger.info("This is an info")
logger.warning("This is a warning")
logger.error("This is an error")

20

[INFO] __main__: This is an info
[WARNING] __main__: This is a warning
[ERROR] __main__: This is an error


In [ ]:
def mask(s: str) -> str:
    return "*" * len(s)


assert mask("test") == "****"

In [ ]:
# | exporti


def _get_credentials(
    username: Optional[str] = None, password: Optional[str] = None
) -> Tuple[(str, str)]:
    """Returns the value for username and password.

    If username is **None**, retrive the value from AIRT_SERVICE_USERNAME environment variable.
    If password is **None**, retrive the value from AIRT_SERVICE_PASSWORD environment variable.

    Args:
        username: Username for your developer account.
        password: Password for your developer account.

    Returns:
        The values for username and password as a tuple.

    Raises:
        Key Error, if the environment variables are not set.
    """

    username = username if username is not None else os.environ.get(SERVICE_USERNAME)

    password = password if password is not None else os.environ.get(SERVICE_PASSWORD)

    if not username and not password:
        raise KeyError(
            f"The username and password are neither passed as parameters nor set in the environment variables "
            f"`{SERVICE_USERNAME}` and `{SERVICE_PASSWORD}`."
        )

    elif not username:
        raise KeyError(
            f"The username is neither passed as parameter nor set in the environment variable {SERVICE_USERNAME}."
        )

    elif not password:
        raise KeyError(
            f"The password is neither passed as parameter nor set in the environment variable {SERVICE_PASSWORD}."
        )

    return (username, password)

In [ ]:
username = "fake_username"
password = "fake_password"

expected = ("fake_username", "fake_password")

actual = _get_credentials(username, password)

display(actual)
assert actual == expected

('fake_username', 'fake_password')

In [ ]:
# Assign env vars to temp variables and delete the env vars
if os.environ.get(SERVICE_USERNAME):
    airt_service_username = os.environ.get(SERVICE_USERNAME)
    del os.environ[SERVICE_USERNAME]

if os.environ.get(SERVICE_PASSWORD):
    airt_service_password = os.environ.get(SERVICE_PASSWORD)
    del os.environ[SERVICE_PASSWORD]

with pytest.raises(KeyError) as e:
    _get_credentials()

display(f"{e.value=}")
assert (
    f"'The username and password are neither passed as parameters nor set in the environment variables `{SERVICE_USERNAME}` and `{SERVICE_PASSWORD}`.'"
    == str(e.value)
)

"e.value=KeyError('The username and password are neither passed as parameters nor set in the environment variables `AIRT_SERVICE_USERNAME` and `AIRT_SERVICE_PASSWORD`.')"

In [ ]:
# Assign only username in env var
os.environ[SERVICE_USERNAME] = airt_service_username

with pytest.raises(KeyError) as e:
    _get_credentials()

display(f"{e.value=}")
assert (
    f"'The password is neither passed as parameter nor set in the environment variable {SERVICE_PASSWORD}.'"
    == str(e.value)
)

"e.value=KeyError('The password is neither passed as parameter nor set in the environment variable AIRT_SERVICE_PASSWORD.')"

In [ ]:
# assign airt_service_password in env var
os.environ[SERVICE_PASSWORD] = airt_service_password

# deleting airt_service_username in env var
if os.environ.get(SERVICE_USERNAME):
    del os.environ[SERVICE_USERNAME]

with pytest.raises(KeyError) as e:
    _get_credentials()

display(f"{e.value=}")
assert (
    f"'The username is neither passed as parameter nor set in the environment variable {SERVICE_USERNAME}.'"
    == str(e.value)
)

# resetting the username and password env variables
os.environ[SERVICE_USERNAME] = airt_service_username
assert os.environ[SERVICE_USERNAME] == airt_service_username
assert os.environ[SERVICE_PASSWORD] == airt_service_password

"e.value=KeyError('The username is neither passed as parameter nor set in the environment variable AIRT_SERVICE_USERNAME.')"

In [ ]:
# | export


@export("airt.client")
class Client:
    """A class for authenticating and accessing the airt service.

    To access the airt service, you must first create a developer account. To obtain one, please contact us at [info@airt.ai](mailto:info@airt.ai).

    After successful verification, you will receive an email with the username and password for the developer account.

    Once you have the credentials, use them to get an access token by calling `get_token` method. It is necessary to get
    an access token; otherwise, you won't be able to access all of the airt service's APIs. You can either pass the username, password, and server
    address as parameters to the `get_token` method or store them in the environment variables **AIRT_SERVICE_USERNAME**,
    **AIRT_SERVICE_PASSWORD**, and **AIRT_SERVER_URL**.

    In addition to the regular authentication with credentials, you can also enable multi-factor authentication (MFA) and single sign-on (SSO)
    for generating tokens.

    To help protect your account, we recommend that you enable multi-factor authentication (MFA). MFA provides additional security by requiring
    you to provide unique verification code (OTP) in addition to your regular sign-in credentials when performing critical operations.

    Your account can be configured for MFA in just two easy steps:

    1. To begin, you need to enable MFA for your account by calling the `User.enable_mfa` method, which will generate a QR code. You can then
    scan the QR code with an authenticator app, such as Google Authenticator and follow the on-device instructions to finish the setup in your smartphone.

    2. Finally, activate MFA for your account by calling `User.activate_mfa` and passing the dynamically generated six-digit verification code from your
    smartphone's authenticator app.

    After activating MFA for your account, you must pass the dynamically generated six-digit verification code, along with your username and password,
    to the `get_token` method to generate new tokens.

    Single sign-on (SSO) can be enabled for your account in three simple steps:

    1. Enable the SSO for a provider by calling the `User.enable_sso` method with the SSO provider name and an email address. At the moment,
    we only support **"google"** and **"github"** as SSO providers. We intend to support additional SSO providers in future releases.

    2. Before you can start generating new tokens with SSO, you must first authenticate with the SSO provider. Call the `get_token` with
    the same SSO provider you have enabled in the step above to generate an SSO authorization URL. Please copy and paste it into your
    preferred browser and complete the authentication process with the SSO provider.

    3. After successfully authenticating with the SSO provider, call the `set_sso_token` method to generate a new token and use it automatically
    in all future interactions with the airt server.

    Here's an example of how to use the Client class to authenticate and display the details of the currently logged-in user.

    Example:
        ```python
        # Importing necessary libraries
        from  airt.client import Client, User

        # Authenticate
        # MFA enabled users must pass the OTP along with the username and password
        # to the get_token method.
        Client.get_token(username="{fill in username}", password="{fill in password}")

        # Print the logged-in user details
        print(User.details())
        ```
    """

    server: Optional[str] = None
    auth_token: Optional[str] = None
    sso_authorization_url: Optional[str] = None

    def __init__(
        self,
        server: str,
        auth_token: str,
        sso_authorization_url: Optional[str] = None,
    ):
        Client.server = server
        Client.auth_token = auth_token
        Client.sso_authorization_url = sso_authorization_url

    @classmethod
    def get_token(  # type: ignore
        cls,
        *,
        username: Optional[str] = None,
        password: Optional[str] = None,
        server: Optional[str] = None,
        sso_provider: Optional[str] = None,
        otp: Optional[str] = None,
    ) -> Optional[str]:
        """Get application token for airt service from a username/password pair.

        This methods validates the developer credentials and returns an auth token. The returned auth
        token is implicitly used in all the interactions with the server.

        If you've already enabled multi-factor authentication (MFA) for your account, you'll need to
        pass the dynamically generated six-digit verification code along with your username and
        password to generate new tokens.

        If the token is requested using Single sign-on (SSO), an authorization URL will be returned.
        Please copy and paste it into your preferred browser and complete the SSO provider
        authentication within 10 minutes. Otherwise, the SSO login will time out and you will need
        to re-request the token.

        Args:
            username: Username for the developer account. If None (default value), then the value from
                **AIRT_SERVICE_USERNAME** environment variable is used.
            password: Password for the developer account. If None (default value), then the value from
                **AIRT_SERVICE_PASSWORD** environment variable is used.
            server: The airt server uri. If None (default value), then the value from **AIRT_SERVER_URL** environment variable
                is used. If the variable is not set as well, then the default public server will be used. Please leave this
                setting to default unless you are running the service in your own server (please email us to info@airt.ai
                for that possibility).
            sso_provider: Name of the Single sign-on (SSO) provider. Please pass this parameter only if you have successfully
                enabled SSO for this provider. At present, the API only supports "google" and "github" as valid SSO providers.
            otp: Dynamically generated six-digit verification code from the authenticator app or the OTP you have received via SMS.

        Returns:
            The authorization url if the token is requested using Single sign-on (SSO).

        Raises:
            ValueError: If the username/password pair does not match.
            ConnectionError: If the server address is invalid or not reachable.
            KeyError: If username/password is neither passed as parameters nor stored in environment variables.

        Here's an example of a non-MFA user authenticating and generating a new token

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import User, Client

            # Authenticate
            Client.get_token(username="{fill in username}", password="{fill in password}")

            # Print the logged-in user details
            print(User.details())
            ```

        Here's an example of a MFA user authenticating using SMS OTP and generating a new token

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, User

            # Request OTP via SMS to authenticate
            # If you want to use the OTP from the authenticator app, skip this step and
            # don't generate an SMS OTP; instead, pass the OTP from the authenticator
            # app to the get_token method below
            username="{fill in username}"
            User.send_sms_otp(
                username=username,
                message_template_name="get_token" # Don't change the message_template_name
            )

            # Authenticate using SMS OTP
            # The send_sms_otp method will send the OTP via SMS to the registered
            # phone number, which you must fill below
            password="{fill in password}"
            otp="{fill in otp}"
            Client.get_token(username=username, password=password, otp=otp)

            # Print the logged-in user details
            print(User.details())
            ```
        """
        cls.server = get_base_url(server)

        username, password = _get_credentials(username, password)

        if otp is not None:
            password = json.dumps({"password": password, "user_otp": otp})

        if sso_provider is None:
            response = post_data(
                url=f"{cls.server}/token",
                data=dict(username=username, password=password),
                token=None,
            )

            cls.auth_token = response["access_token"]
        else:
            response = post_data(
                url=f"{cls.server}/sso/initiate",
                data=json.dumps(  # type: ignore
                    dict(
                        username=username, password=password, sso_provider=sso_provider
                    )
                ),
                token=None,
            )

            cls.sso_authorization_url = response["authorization_url"]
            return cls.sso_authorization_url

    @classmethod
    def set_sso_token(cls):
        """Set the application token generated using Single sign-on (SSO).

        The token set using this method will be implicitly used in all the interactions with the server.

        Please call this method only if you successfully enabled and completed the login with the Single
        sign-on (SSO) provider. If not, please call the `get_token` method with an appropriate
        sso_provider to initiate the SSO authentication.

        Here's an example of authenticating with Single sign-on (SSO) using google and setting the
        newly generated token to interact with the airt service.

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, User

            # Authenticate
            Client.get_token(username="{fill in username}", password="{fill in password}")

            # Enable single sign-on (SSO) and use google as the provider
            sso_provider="google"
            sso_email="{fill in sso_email}"
            User.enable_sso(sso_provider=sso_provider, sso_email=sso_email)

            # Authenticate using Single sign-on (SSO)
            # To generate a token using SSO, you must first authenticate with the provider.
            # The command below will generate an authorization URL for you.
            # Please copy and paste it into your preferred browser and complete the
            # SSO provider authentication within 10 minutes. Otherwise, the SSO login
            # will time out and you will need to call the get_token method again.
            sso_url = Client.get_token(sso_provider=sso_provider)
            print(sso_url)

            # Once the provider authentication is successful, call the below method to
            # set the generated token
            Client.set_sso_token()

            # If set_sso_token fails, the line below will throw an error.
            print(User.details())
            ```
        """
        quoted_authorization_url = urllib.parse.quote(cls.sso_authorization_url)
        response = get_data(
            url=f"{cls.server}/sso/token/?authorization_url={quoted_authorization_url}",
            token=None,
        )

        cls.auth_token = response["access_token"]

    @classmethod
    def set_token(cls, token: Optional[str] = None, server: Optional[str] = None):
        """Set application token for airt service.

        If you already have a valid token, you can call this method to set it and use it in all
        subsequent interactions with the airt server.

        Please call this method only if you already have a token. If not, please call the `get_token` method to generate one.

        Args:
            token: The application token obtained by calling the `get_token` method, or an APIKey obtained by calling
                the `APIKey.create` method. If None (default value), then the value from **AIRT_SERVICE_TOKEN** environment variable is used.
            server: The airt server uri. If None (default value), then the value from **AIRT_SERVER_URL** environment variable
                is used. If the variable is not set as well, then the default public server will be used. Please leave this
                setting to default unless you are running the service in your own server (please email us to info@airt.ai
                for that possibility).

        An example to set an existing token:

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, User

            # Optional Step: For demonstration purpose, generate a new token
            # When you generate a new token with the get_token method, you do not
            # need to explicitly call set_token. It is shown here for demo purposes only.
            # Skip this step if you already have a valid token and pass it directly to
            # the set_token method below
            Client.get_token(username="{fill in username}", password="{fill in password}")

            # Setting a valid token
            Client.set_token(token=Client.auth_token)

            # If set_token fails, the line below will throw an error.
            print(User.details())
            ```
        """

        auth_token = token if token is not None else os.environ.get(SERVICE_TOKEN)

        if not auth_token:
            raise KeyError(
                f"The token is neither passed as parameter nor set in the environment variable {SERVICE_TOKEN}."
            )

        cls.auth_token = auth_token
        cls.server = get_base_url(server)

    @staticmethod
    def version() -> dict:
        """Return the client and server versions.

        Returns:
            A dict containing the client and server versions.

        Raises:
            ConnectionError: If the server address is invalid or not reachable.

        An example to get the client and server versions:

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client

            # Get the client and server versions
            print(Client.version())
            ```
        """

        response = Client._get_data(relative_url=f"/version")

        version = {
            # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
            "client": importlib.import_module(CLIENT_NAME).__version__,
            "server": response["airt_service"],
        }

        return version

    @classmethod
    def _get_server_url_and_token(cls) -> Tuple[Optional[str], Optional[str]]:
        """Fetch the server URL and the auth token.

        Returns:
            A tuple containing server URL and auth token.
        """

        cls.server = get_base_url(cls.server)

        return cls.server, cls.auth_token

    @classmethod
    def _post_data(
        cls,
        relative_url: str,
        data: Optional[Dict[str, Any]] = None,
        json: Optional[Dict[str, Any]] = None,
    ) -> Dict[str, Any]:
        """Make a POST request.

        This method will implicitly add the server base URL and the token for every request.

        Args:
            relative_url: The relative URL of the server's API endpoint.
            data: A Dictionary object to send in the body of the POST request. The data sent in this param will automatically be form-encoded by the request library.
            json: A Dictionary object to send in the body of the POST request. The data sent in this param will automatically be JSON-encoded by the request library.

        Returns:
            Response body as a dictionary.

        Raises:
            ConnectionError: If the server is not reachable.
            ValueError: If the response code is not in range of 200 - 399.
        """

        server, auth_token = Client._get_server_url_and_token()

        return post_data(
            url=f"{server}{relative_url}",
            data=data,
            json=json,
            token=auth_token,
        )

    @classmethod
    def _get_data(cls, relative_url: str) -> Any:
        """Make a GET request.

        This method will implicitly add the server base URL and the token for every request.

        Args:
            relative_url: The relative URL of the API endpoint.

        Returns:
            A dictionary that encapsulates the response body.

        Raises:
            ConnectionError: If the server is not reachable.
            ValueError: If the response code is not in range of 200 - 399.
        """

        server, auth_token = Client._get_server_url_and_token()

        return get_data(url=f"{server}{relative_url}", token=auth_token)

    @classmethod
    def _delete_data(cls, relative_url: str) -> Dict[str, Any]:
        """Make a DELETE request.

        This method will implicitly add the server base URL and the token for every request.

        Args:
            relative_url: The relative URL of the API endpoint.

        Returns:
            A dictionary that encapsulates the response body.

        Raises:
            ConnectionError: If the server is not reachable.
            ValueError: If the response code is not in range of 200 - 399.
        """

        server, auth_token = Client._get_server_url_and_token()

        return delete_data(url=f"{server}{relative_url}", token=auth_token)

In [ ]:
# Run example for Client
username = os.environ[SERVICE_USERNAME]
password = os.environ[SERVICE_PASSWORD]

run_examples_from_docstring(
    Client,
    username=username,
    password=password,
)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, User                                                                       │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     # MFA enabled users must pass the OTP along with the username and password                                  │
│     # to the get_token method.                                                                                  │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Print the logged-in user details                                                                          │
│     print(User.details())                                                                                       │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ {'username': 'johndoe', 'first_name': 'John', 'last_name': 'Doe', 'email': 'johndoe@airt.ai',               │ │
│ │ 'subscription_type': 'small', 'super_user': False, 'phone_number': None, 'sso_profile_pic_url': None,       │ │
│ │ 'uuid': 'e402909e-b462-4885-96d4-1996ec1af7e1', 'disabled': False, 'created': '2023-05-29T03:35:45',        │ │
│ │ 'is_phone_number_verified': False, 'is_mfa_active': False}                                                  │ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── stderr ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# Run example for Client.get_token
# The below example will throw Incorrect username or password, thats expected because
# if we send valid username and password, the sms will be sent to the passed phone number.
# We don't want to send the SMS each time we run the tests

username = os.environ[SERVICE_USERNAME]
password = os.environ[SERVICE_PASSWORD]
with pytest.raises(RuntimeError) as e:
    run_examples_from_docstring(
        Client.get_token,
        username="invalid_username",
        password="invalid_username",
        otp="000000",
        supress_stderr=True,
    )

<module>:15: No type or annotation for parameter 'username'
<module>:17: No type or annotation for parameter 'password'
<module>:19: No type or annotation for parameter 'server'
<module>:23: No type or annotation for parameter 'sso_provider'
<module>:25: No type or annotation for parameter 'otp'
<module>:28: No type or annotation for returned value 1


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import User, Client                                                                       │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Print the logged-in user details                                                                          │
│     print(User.details())                                                                                       │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── stderr supressed ──────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# Run example for Client.set_sso_token
with pytest.raises(RuntimeError) as e:
    run_examples_from_docstring(
        Client.set_sso_token,
        username="invalid_username",
        password="invalid_username",
        sso_email="sso_email@mail.com",
        supress_stderr=True,
    )

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, User                                                                       │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Enable single sign-on (SSO) and use google as the provider                                                │
│     sso_provider="google"                                                                                       │
│     sso_email="{fill in sso_email}"                                                                             │
│     User.enable_sso(sso_provider=sso_provider, sso_email=sso_email)                                             │
│                                                                                                                 │
│     # Authenticate using Single sign-on (SSO)                                                                   │
│     # To generate a token using SSO, you must first authenticate with the provider.                             │
│     # The command below will generate an authorization URL for you.                                             │
│     # Please copy and paste it into your preferred browser and complete the                                     │
│     # SSO provider authentication within 10 minutes. Otherwise, the SSO login                                   │
│     # will time out and you will need to call the get_token method again.                                       │
│     sso_url = Client.get_token(sso_provider=sso_provider)                                                       │
│     print(sso_url)                                                                                              │
│                                                                                                                 │
│     # Once the provider authentication is successful, call the below method to                                  │
│     # set the generated token                                                                                   │
│     Client.set_sso_token()                                                                                      │
│                                                                                                                 │
│     # If set_sso_token fails, the line below will throw an error.                                               │
│     print(User.details())                                                                                       │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── stderr supressed ──────────────────────────────────────────────╮ │
│ │                                                     

In [ ]:
# Run example for Client.version

run_examples_from_docstring(
    Client.version,
)

<module>:3: No type or annotation for returned value 1


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client                                                                             │
│                                                                                                                 │
│     # Get the client and server versions                                                                        │
│     print(Client.version())                                                                                     │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ {'client': '2023.4.0rc0', 'server': '2023.3.0rc0'}                                                          │ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── stderr ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# Run example for Client.set_token

run_examples_from_docstring(Client.set_token, username=username, password=password)

<module>:8: No type or annotation for parameter 'token'
<module>:10: No type or annotation for parameter 'server'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, User                                                                       │
│                                                                                                                 │
│     # Optional Step: For demonstration purpose, generate a new token                                            │
│     # When you generate a new token with the get_token method, you do not                                       │
│     # need to explicitly call set_token. It is shown here for demo purposes only.                               │
│     # Skip this step if you already have a valid token and pass it directly to                                  │
│     # the set_token method below                                                                                │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Setting a valid token                                                                                     │
│     Client.set_token(token=Client.auth_token)                                                                   │
│                                                                                                                 │
│     # If set_token fails, the line below will throw an error.                                                   │
│     print(User.details())                                                                                       │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ {'username': 'johndoe', 'first_name': 'John', 'last_name': 'Doe', 'email': 'johndoe@airt.ai',               │ │
│ │ 'subscription_type': 'small', 'super_user': False, 'phone_number': None, 'sso_profile_pic_url': None,       │ │
│ │ 'uuid': 'e402909e-b462-4885-96d4-1996ec1af7e1', 'disabled': False, 'created': '2023-05-29T03:35:45',        │ │
│ │ 'is_phone_number_verified': False, 'is_mfa_active': False}                                                  │ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── stderr ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# Tests for Client._get_server_url_and_token.
# cls.server is set to None, and the env variable is not set. The public URL should be returned

# deleting the env variable
_SERVER_URL = None

if os.environ.get(SERVER_URL):
    _SERVER_URL = os.environ.get(SERVER_URL)
    del os.environ[SERVER_URL]

server, auth_token = Client._get_server_url_and_token()

expected = "https://api.airt.ai"

display(f"{server=}")
assert server == expected

# Assigning the value back to the env variable
if _SERVER_URL:
    os.environ[SERVER_URL] = _SERVER_URL

"server='https://api.airt.ai'"

In [ ]:
# Tests for Client._get_server_url_and_token.
# cls.server is set to "http://example-service:6006", the same should be returned

Client.server = "http://example-service:6006"

server, auth_token = Client._get_server_url_and_token()

expected = "http://example-service:6006"

display(f"{len(server)=} {server=}")
display(f"{len(expected)=}")
assert server == expected

"len(server)=27 server='http://example-service:6006'"

'len(expected)=27'

In [ ]:
# Tests for Client._get_server_url_and_token.
# cls.server is set to None, and getting the server URL from env variable

Client.server = None

server, auth_token = Client._get_server_url_and_token()

expected = os.environ.get(SERVER_URL)

display(f"{len(server)=} {server=}")
display(f"{len(expected)=}")
assert server == expected

"len(server)=24 server='http://airt-service:6006'"

'len(expected)=24'

In [ ]:
# Tests for version

version = Client.version()

display(version)

assert list(version.keys()) == ["client", "server"]

{'client': '2023.4.0rc0', 'server': '2023.3.0rc0'}

In [ ]:
# Tests for Client.get_token
# Testing the SSO flow
# Negative scenario: generating SSO token without enabling the SSO provider
sso_providers = ["google", "github"]
for sso_provider in sso_providers:
    with pytest.raises(ValueError) as e:
        Client.get_token(sso_provider=sso_provider)
    assert "SSO is not enabled" in str(e.value)
    display(str(e.value))

'SSO is not enabled for the provider.'

'SSO is not enabled for the provider.'

In [ ]:
# A helper context manager to create new users for testing
@contextmanager
def create_normal_user_for_testing():
    try:
        # 1. Get Super user token
        username = os.environ[SERVICE_SUPER_USER]
        password = os.environ[SERVICE_PASSWORD]

        Client.get_token(username=username, password=password)

        _user_name = f"random_user_{randrange(10000)}_{randrange(10000)}"
        _email = f"random_user_{randrange(10000)}_{randrange(10000)}@email.com"
        _password = "random_password"

        req_data = dict(
            username=_user_name,
            first_name="random_first_name",
            last_name="random_last_name",
            email=_email,
            password=_password,
            super_user=False,
            subscription_type="test",
        )

        response = Client._post_data(relative_url=f"/user/", json=req_data)

        response_df = pd.DataFrame(response, index=[0])[User.USER_COLS]

        assert response_df.shape == (1, len(User.USER_COLS))

        Client.get_token(username=_user_name, password=_password)

        yield _user_name, _password

    finally:
        pass

In [ ]:
# Tests for Client.get_token
# Testing the SSO flow
# Positive scenario: SSO enabled user generating token using non-SSO flow


with create_normal_user_for_testing() as credentials:
    username = credentials[0]
    password = credentials[1]

    sso_email = "sso_email@mail.com"
    sso_provider = "google"
    actual = Client._post_data(
        relative_url=f"/user/sso/enable",
        json=dict(sso_provider=sso_provider, sso_email=sso_email, otp=None),
    )
    Client.get_token(username=username, password=password)
    server, auth_token = Client._get_server_url_and_token()

    display(f"{server=}, {mask(auth_token)=}")

"server='http://airt-service:6006', mask(auth_token)='*************************************************************************************************************************************************'"

In [ ]:
# Tests for Client.get_token
# Testing the SSO flow
# Positive scenario: SSO enabled user generating token using SSO flow


with create_normal_user_for_testing() as credentials:
    username = credentials[0]
    password = credentials[1]

    sso_email = "random_email@mail.com"
    sso_provider = "google"
    actual = Client._post_data(
        relative_url=f"/user/sso/enable",
        json=dict(sso_provider=sso_provider, sso_email=sso_email, otp=None),
    )
    sso_authorization_url = Client.get_token(
        username=username, password=password, sso_provider=sso_provider
    )
    display(sso_authorization_url)
    assert sso_authorization_url == Client.sso_authorization_url
    assert sso_provider in Client.sso_authorization_url

'https://accounts.google.com/o/oauth2/v2/auth?response_type=code&client_id=842138153914-6kvm51cpin7iocg3nrsnl44s3d24u047.apps.googleusercontent.com&redirect_uri=http%3A%2F%2F127.0.0.1%3A6006%2Fsso%2Fcallback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.profile+openid&state=aa296044481fb9d17a88aa5cdee7fe24d78093ec4cb7e65adb8c4fefc2f697de_random_user_5761_1673&prompt=select_account'

In [ ]:
# Tests for Client.get_token
# Testing the SSO flow
# Positive scenario: SSO enabled user generating token using SSO flow


with create_normal_user_for_testing() as credentials:
    username = credentials[0]
    password = credentials[1]

    sso_email = "random_email@mail.com"
    sso_provider = "github"
    actual = Client._post_data(
        relative_url=f"/user/sso/enable",
        json=dict(sso_provider=sso_provider, sso_email=sso_email, otp=None),
    )
    sso_authorization_url = Client.get_token(
        username=username, password=password, sso_provider=sso_provider
    )
    display(sso_authorization_url)
    assert sso_authorization_url == Client.sso_authorization_url
    assert sso_provider in Client.sso_authorization_url

'https://github.com/login/oauth/authorize?response_type=code&client_id=a0f58d9e50375190dbf0&redirect_uri=http%3A%2F%2F127.0.0.1%3A6006%2Fsso%2Fcallback&scope=user%3Aemail&state=07e7373eccd44059a474c26ccecc1634aca5817ee39c74be9dcd819332d84bef_random_user_1442_90&prompt=select_account'

In [ ]:
# Tests for Client.get_token
# Testing the SSO flow
# Negative scenario: Non-MFA user passing OTP while intiating SSO flow

with create_normal_user_for_testing() as credentials:
    username = credentials[0]
    password = credentials[1]

    sso_email = "random_user@gmail.com"
    sso_provider = "github"
    random_otp = "123456"
    actual = Client._post_data(
        relative_url=f"/user/sso/enable",
        json=dict(sso_provider=sso_provider, sso_email=sso_email, otp=None),
    )
    with pytest.raises(ValueError) as e:
        Client.get_token(sso_provider=sso_provider, otp=random_otp)
    assert "Incorrect username or password" in str(e.value)
str(e.value)

'Incorrect username or password. Please try again.'

In [ ]:
# Tests for Client.get_token
# Testing the SSO flow
# Negative scenario: Generating token for wrong sso_provider


with create_normal_user_for_testing() as credentials:
    username = credentials[0]
    password = credentials[1]

    sso_email = "random_user@gmail.com"
    sso_provider = "github"
    actual = Client._post_data(
        relative_url=f"/user/sso/enable",
        json=dict(sso_provider=sso_provider, sso_email=sso_email, otp=None),
    )
    with pytest.raises(ValueError) as e:
        Client.get_token(sso_provider=sso_provider)
    assert "SSO is not enabled" in str(e.value)
str(e.value)

'SSO is not enabled for the provider.'

In [ ]:
# Tests for Client.get_token
# Testing the SSO flow
# Negative scenario: Generating token for invalid sso_provider


with create_normal_user_for_testing() as credentials:
    username = credentials[0]
    password = credentials[1]

    sso_email = "random_user@gmail.com"
    sso_provider = "google"
    invalid_sso_provider = "invalid_sso_provider"
    actual = Client._post_data(
        relative_url=f"/user/sso/enable",
        json=dict(sso_provider=sso_provider, sso_email=sso_email, otp=None),
    )
    with pytest.raises(ValueError) as e:
        Client.get_token(sso_provider=invalid_sso_provider)
str(e.value)

"Invalid SSO provider. Valid SSO providers are: ['google', 'github']"

In [ ]:
# Tests for Client.set_sso_token
# Negative scenario: Trying to set sso token without completing the SSO authentication

with create_normal_user_for_testing() as credentials:
    username = credentials[0]
    password = credentials[1]

    sso_email = "random_email@mail.com"
    sso_provider = "google"
    actual = Client._post_data(
        relative_url=f"/user/sso/enable",
        json=dict(sso_provider=sso_provider, sso_email=sso_email, otp=None),
    )
    sso_authorization_url = Client.get_token(
        username=username, password=password, sso_provider=sso_provider
    )
    display(sso_authorization_url)

    assert sso_authorization_url == Client.sso_authorization_url

    with pytest.raises(ValueError) as e:
        Client.set_sso_token()

    err = str(e.value)
    assert "SSO authentication is not complete" in err
    display(err)

'https://accounts.google.com/o/oauth2/v2/auth?response_type=code&client_id=842138153914-6kvm51cpin7iocg3nrsnl44s3d24u047.apps.googleusercontent.com&redirect_uri=http%3A%2F%2F127.0.0.1%3A6006%2Fsso%2Fcallback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.profile+openid&state=b379ce2e66de1200d5d26df74f9d051cd678875425b6476abe47cb23b9fb3c3c_random_user_5979_620&prompt=select_account'

'SSO authentication is not complete. Please click on the authentication link you have received while requesting a new token and complete the login process first.'

In [ ]:
# Tests for Client
# Checking positive scenario. Read both username and password from the environment variables
Client.get_token()
server, auth_token = Client._get_server_url_and_token()

display(f"{server=}, {mask(auth_token)=}")

assert server == os.environ.get(SERVER_URL)
assert len(auth_token) >= 127  # maybe

"server='http://airt-service:6006', mask(auth_token)='*******************************************************************************************************************************'"

In [ ]:
# Tests for Client
# Checking positive scenario. Passing all the required parameters in arguments

username = os.environ[SERVICE_USERNAME]
password = os.environ[SERVICE_PASSWORD]

Client.get_token(username=username, password=password)

server, auth_token = Client._get_server_url_and_token()
display(f"{server=}, {mask(auth_token)=}")

assert server == os.environ.get(SERVER_URL)
assert len(auth_token) >= 127  # maybe

"server='http://airt-service:6006', mask(auth_token)='*******************************************************************************************************************************'"

In [ ]:
# Tests for Client
# Checking negative scenario. Passing wrong username and password combination

username = "randomUser"
password = "whatever"

with pytest.raises(ValueError) as e:
    Client.get_token(username=username, password=password)

display(f"{e.value=}")
assert "Incorrect username or password" in str(e.value)

"e.value=ValueError('Incorrect username or password. Please try again.')"

In [ ]:
# Test cases for Client._get_data
# Checking positive scenario.

response = Client._get_data(relative_url=f"/version")

display(f"{response=}")
assert "airt_service" in response.keys()

"response={'airt_service': '2023.3.0rc0'}"

In [ ]:
# Test cases for Client._delete_data


def create_and_return_test_db_id():
    uri = "s3://test-airt-service/account_312571_events_csv"
    access_key = os.environ["AWS_ACCESS_KEY_ID"]
    secret_key = os.environ["AWS_SECRET_ACCESS_KEY"]
    tag = None

    response = Client._post_data(
        relative_url="/datablob/from_s3",
        json=dict(uri=uri, access_key=access_key, secret_key=secret_key, tag=tag),
    )

    return response["uuid"]


db_id = create_and_return_test_db_id()

# Deleting the sample datablob
response = Client._delete_data(relative_url=f"/datablob/{db_id}")
assert response["uuid"] == db_id

# Negative scenario. Trying to delete already deleted data source
with pytest.raises(ValueError) as e:
    Client._delete_data(relative_url=f"/datablob/{db_id}")

display(f"\n{e.value=}")

"\ne.value=ValueError('The datablob has already been deleted.')"

In [ ]:
# Test cases for Client.set_token
# Negative case: The token is not passed as parameter nor set in SERVICE_TOKEN

with pytest.raises(KeyError) as e:
    Client.set_token()

display(f"{str(e.value)=}")
assert (
    str(e.value)
    == f"'The token is neither passed as parameter nor set in the environment variable {SERVICE_TOKEN}.'"
)

'str(e.value)="\'The token is neither passed as parameter nor set in the environment variable AIRT_SERVICE_TOKEN.\'"'

In [ ]:
# Test cases for Client.set_token
# Positive case: Setting valid token in SERVICE_TOKEN env variable and accessing the API

os.environ[SERVICE_TOKEN] = auth_token

Client.set_token()

response = Client._get_data(
    relative_url=f"/datablob/?disabled=false&completed=false&offset=0&limit=100"
)

display(f"{type(response)=}")
assert isinstance(response, list)

# Deleting the env variable
del os.environ[SERVICE_TOKEN]
assert not os.environ.get(SERVICE_TOKEN)

"type(response)=<class 'list'>"

In [ ]:
# Test cases for Client.set_token
# Positive case: Setting token and server using Client.set_token and asserting the same using Client._get_server_url_and_token

fake_token = "fake-token"
fake_server = "http://fake-server"

Client.set_token(fake_token, fake_server)

server, auth_token = Client._get_server_url_and_token()

display(f"{server=}, {auth_token=}")
assert (server, auth_token) == (fake_server, fake_token)

"server='http://fake-server', auth_token='fake-token'"

In [ ]:
# Test cases for Client.set_token
# Negative case: Setting invalid token and accessing the API

Client.set_token(fake_token)

with pytest.raises(ValueError) as e:
    Client._get_data(
        relative_url=f"/datablob/?disabled=false&completed=false&offset=0&limit=100"
    )


display(f"{str(e.value)=}")

"str(e.value)='Your credentials could not be validated. The developer token/apikey is invalid or expired.'"

In [ ]:
# Test cases for Client.set_token
# Positive case: Setting valid token and accessing the API

Client.get_token()
server, auth_token = Client._get_server_url_and_token()

Client.set_token(auth_token)

response = Client._get_data(
    relative_url=f"/datablob/?disabled=false&completed=false&offset=0&limit=100"
)

display(f"{type(response)=}")
assert isinstance(response, list)

"type(response)=<class 'list'>"